In [ ]:
!pip install -q langchain huggingface-hub faiss-cpu transformers sentence-transformers
!pip install -q newsapi-python
!pip install -q langchain-community
!pip install -q openai==0.28
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currentl

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `LLM` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM`


In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from langchain.vectorstores import FAISS
from faiss import IndexFlatIP
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.embeddings import SentenceTransformerEmbeddings
import requests
from bs4 import BeautifulSoup
from newsapi import NewsApiClient
from huggingface_hub import InferenceClient
import openai
from langchain_core.documents import Document
from uuid import uuid4
from langchain_community.vectorstores.utils import DistanceStrategy
import time

In [ ]:
class NormalizedEmbeddings:
    def __init__(self, base_model):
        self.base_model = base_model

    def embed_query(self, query):
        embedding = np.array(self.base_model.embed_query(query))
        return embedding / np.linalg.norm(embedding)

    def embed_documents(self, documents):
        embeddings = np.array(self.base_model.embed_documents(documents))
        return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [ ]:
class DualRagModel:

    #TODO: implement GPT and Mamba options for model
    def __init__(self, model='llama', logging='quiet'):
        """
        model parameter can be 'llama', 'gpt', or 'mamba'

        'quiet' logging means no logging at all
        'minimal' logging shows AAAAAA, BBBBBB, or CCCCCC and also GPT-created queries
        'verbose' logging shows logs from 'minimal', GPT summaries, context and knowledge scores, and the number of factual and contextual results being used
        """
        self.logging = logging
        self.newsapi = NewsApiClient(api_key= ) # ENTER NEWSAPI API KEY HERE
        self.seen_urls = set()

        embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
        self.normalized_embeddings = NormalizedEmbeddings(embeddings)
        dimensions: int = len(embeddings.embed_query("dummy"))

        self.knowledge_db = FAISS(
            embedding_function=self.normalized_embeddings.embed_query,
            index=IndexFlatIP(dimensions),  # Use inner product for cosine similarity
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
            distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT
        )

        self.user_specific_db = FAISS(
            embedding_function=self.normalized_embeddings.embed_query,
            index=IndexFlatIP(dimensions),  # Use inner product for cosine similarity
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
            distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT
        )

        self.model = model

        if model == 'llama':
            self.client = InferenceClient(api_key=) # ENTER HUGGINGFACE API KEY HERE

    def news_search(self, query):
        data = self.newsapi.get_everything(q=query,
                                           language='en',
                                           sort_by='relevancy',
                                           page_size=5)
        # print("News search: ", data['articles'])
        return data['articles']

    def fetch_full_content(self, url):
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')
            return '\n'.join([p.get_text() for p in paragraphs])
        except Exception as e:
            return None  # Return None if full content cannot be fetched


    def summarize_content_openAI(self, content):
        openai.api_key = # ENTER OPENAI API KEY HERE

        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",  # Specify the model
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Summarize the content below in 150 words or less:\n{content}"}
                ],
                temperature=0.7,
                max_tokens=150
            )
            summary = response['choices'][0]['message']['content']
            return summary
        except Exception as e:
            return f"An error occurred: {e}"

    def update_knowledge_db(self, query):
        articles = self.news_search(query)
        documents = []

        for article in articles:
            if article['url'] not in self.seen_urls:
                self.seen_urls.add(article['url'])
                content = self.fetch_full_content(article['url'])
                if content:
                    summarized_content = self.summarize_content_openAI(content)

                    if self.logging == 'verbose':
                        print("GPT summary: ", summarized_content)

                    documents.append(Document(page_content=summarized_content, metadata={'source': 'news_api', 'timestamp': article['publishedAt'], 'search_timestamp': time.time()}))


        uuids = [str(uuid4()) for _ in range(len(documents))]

        if documents:
            self.knowledge_db.add_documents(documents=documents, ids=uuids)
            print("Knowledge database updated successfully!")

    def adaptive_retrieval(self, query):
        similar_context = self.user_specific_db.similarity_search_with_score(query, k=5)

        if similar_context:
            most_similar_context = similar_context[0] # Document object, similarity score
            for doc, score in similar_context:
                if score > 0.3 and doc.metadata['interaction_timestamp'] > most_similar_context[0].metadata['interaction_timestamp'] + 2:
                    most_similar_context = (doc, score)

            if most_similar_context[1] < 0.25: #be more generous with knowledge DB score threshold with poor context similarity (low similarity score)
                if self.logging != 'quiet':
                    print("AAAAAA")
                print("query is: ", query)
                self.update_knowledge_db(query)

                if self.logging == 'verbose':
                    print("Most similar context score: ", most_similar_context[1])
                    test = self.knowledge_db.similarity_search_with_score(query, score_threshold=0.0, k=10)
                    print("Knowledge scores: ", [x[1] for x in test])

                factual_results = [x[0] for x in self.knowledge_db.similarity_search_with_score(query, score_threshold=0.25, k=10)]
            else:
                query = openai.ChatCompletion.create(
                    model="gpt-4o-mini",
                    messages=[
                        # {"role": "system", "content": "You are forming a more detailed question based on vague question and context."},
                        {"role": "system", "content": "Based on the context and query, generate up to 3 key words."},
                        {"role": "user", "content": f"Combine {query} with context from {most_similar_context[0]}"}
                    ],
                    temperature=0.7,
                    max_tokens=15
                )
                query = query['choices'][0]['message']['content']

                if self.logging != 'quiet':
                    print("BBBBBB")
                    print("GPT-created query: ", query)

                print("query is: ", query)
                self.update_knowledge_db(query)

                if self.logging == 'verbose':
                    print("Most similar context score: ", most_similar_context[1])
                    test = self.knowledge_db.similarity_search_with_score(query, score_threshold=0.0, k=10)
                    print("Knowledge scores: ", [x[1] for x in test])

                # factual_results = [x[0] for x in self.knowledge_db.similarity_search_with_score(query, score_threshold=0.4, k=10)]
                sorted_factual_results = sorted(self.knowledge_db.similarity_search_with_score(query, score_threshold=0.4, k=10),
                                     key=lambda x: x[0].metadata["search_timestamp"],
                                     reverse=True)
                factual_results = [x[0] for x in sorted_factual_results]
        else:
            if self.logging != 'quiet':
                print("CCCCCC")

            # print("query is: ", query)
            self.update_knowledge_db(query)

            if self.logging == 'verbose':
                test = self.knowledge_db.similarity_search_with_score(query, score_threshold=0.0, k=10)
                print("Knowledge scores: ", [x[1] for x in test])

            factual_results = [x[0] for x in self.knowledge_db.similarity_search_with_score(query, score_threshold=0.25, k=10)]

        contextual_results = [x[0] for x in self.user_specific_db.similarity_search_with_score(query, score_threshold=0.3, k=10)]

        if self.logging == 'verbose':
            print("# of factual results:" , len(factual_results))
            print("# of contextual results: ", len(contextual_results))

        results = factual_results + contextual_results

        return results

    def update_user_memory(self, user_query, model_response):
        """
        Updates the user-specific database with the latest query and response.

        Args:
            user_query (str): The query provided by the user.
            model_response (str): The response generated by the model.
            user_specific_db (FAISS): The user-specific vector database.
        """
        documents = []
        documents.append(Document(page_content=user_query + "\n" + model_response, metadata={'source': 'user_query', 'interaction_timestamp': time.time()}))
        uuids = [str(uuid4()) for _ in range(len(documents))]

        self.user_specific_db.add_documents(documents=documents, ids=uuids)
        print("User-specific memory updated successfully!")

    def generate_response(self, query):
        relevant_results = self.adaptive_retrieval(query)
        context = " ".join([result.page_content for result in relevant_results])

        if self.model == 'llama':
            input_prompt = f"Based on this context: {context}\n\nAnswer this question: {query}"

            messages = [
                {
                    "role": "user",
                    "content": input_prompt
                }
            ]

            completion = self.client.chat.completions.create(
                model="meta-llama/Meta-Llama-3-8B-Instruct",
                messages=messages,
                max_tokens=50
            )

            generated_response = completion.choices[0].message.content
            self.update_user_memory(query, generated_response)
            return generated_response

In [ ]:
model = DualRagModel(logging='quiet')

In [ ]:
user_query_1 = 'who shot the UHC ceo'
model_response_1 = model.generate_response(user_query_1)

Knowledge database updated successfully!
User-specific memory updated successfully!


In [ ]:
print(model_response_1)

Luigi Mangione, a 26-year-old Ivy League graduate, was arrested in connection with the shooting death of UnitedHealthcare CEO Brian Thompson in New York City.


In [ ]:
user_query_2 = 'where was he shot'
model_response_2 = model.generate_response(user_query_2)

query is:  Shooting, UnitedHealthcare, Brian Thompson
Knowledge database updated successfully!
User-specific memory updated successfully!


In [ ]:
print(model_response_2)

According to the text, Brian Thompson, the CEO of UnitedHealthcare, was shot outside a hotel, specifically the Hilton Hotel, in Midtown Manhattan, New York City.


In [ ]:
user_query_3 = 'where was the suspect found'
model_response_3 = model.generate_response(user_query_3)

query is:  Suspect, Shot, Hilton Hotel
Knowledge database updated successfully!
User-specific memory updated successfully!


In [ ]:
print(model_response_3)

The suspect, Luigi Mangione, was found in Altoona, Pennsylvania, after a week on the run.


In [ ]:
user_query_4 = 'how did people react'
model_response_4 = model.generate_response(user_query_4)

query is:  how did people react
Knowledge database updated successfully!
User-specific memory updated successfully!


In [ ]:
print(model_response_4)

According to the text, people reacted with dark humor and scathing commentary on social media, sharing memes and jokes about the healthcare industry, and expressing a sense of schadenfreude (pleasure derived from someone else's misfortune) by equating


In [ ]:
user_query_5 = 'when was president yoon impeached'
model_response_5 = model.generate_response(user_query_5)

CCCCCC
GPT summary:  South Korean lawmakers have impeached President Yoon Suk Yeol following his controversial attempt to impose martial law, which led to widespread protests. The impeachment motion passed with support from some members of Yoon's own People Power Party (PPP), despite a prior boycott of the vote. Thousands of anti-Yoon protesters celebrated outside the National Assembly, highlighting significant public discontent, with polls indicating that three-quarters of South Koreans favored Yoon's removal. Yoon, who has been suspended, declared his intention to fight the impeachment, framing the vote as a temporary setback. The constitutional court now has 180 days to decide on the impeachment's validity. If upheld, a presidential election must be held within 60 days. Prime Minister Han Duck-soo
GPT summary:  South Korean police are raiding the presidential office of Yoon Suk Yeol following his failed attempt to impose martial law. While they have accessed civil service offices, t

In [ ]:
print(model_response_5)

According to the context, the impeachment of President Yoon Suk Yeol was not explicitly stated as having occurred at a specific time, but rather that the impeachment motion passed with support from some members of Yoon's own party, despite a prior boycott of the vote. It was mentioned that Yoon has been suspended, and the constitutional court now has 180 days to decide on the impeachment's validity.


In [ ]:
user_query_6 = 'who impeached him'
model_response_6 = model.generate_response(user_query_6)

BBBBBB
GPT-created query:  Who were the members of President Yoon Suk Yeol's own party that supported the impeachment motion against him?
Most similar context score:  0.5214876
Knowledge scores:  [0.6454982, 0.5223322, 0.46192974, 0.43618298, 0.41040105]
# of factual results: 5
# of contextual results:  1
User-specific memory updated successfully!


In [ ]:
print(model_response_6)

According to the context, President Yoon Suk Yeol was impeached by South Korean lawmakers.


In [ ]:
user_query_7 = 'why was he impeached'
model_response_7 = model.generate_response(user_query_7)

BBBBBB
GPT-created query:  Why was President Yoon Suk Yeol impeached by South Korean lawmakers?
Most similar context score:  0.4836761
Knowledge scores:  [0.74355704, 0.655087, 0.6437101, 0.5566671, 0.5318]
# of factual results: 5
# of contextual results:  2
User-specific memory updated successfully!


In [ ]:
print(model_response_7)

According to the context, President Yoon Suk Yeol was impeached by South Korean lawmakers following his controversial attempt to impose martial law, which led to widespread protests and calls for his removal. The impeachment is also mentioned to be a result of a "significant political crisis" and the "first such attempt since the country's shift to democracy in 1987".


In [ ]:
user_query_8 = 'what was the public reaction'
model_response_8 = model.generate_response(user_query_8)

AAAAAA
GPT summary:  On September 6, 2022, Daniel Khalife, a former British Army soldier awaiting trial for espionage, escaped from HMP Wandsworth by hiding under a delivery truck. Using makeshift equipment, he clung to the truck and slipped through security checks, later emerging casually on the road. Khalife had been in contact with Iranian agents, providing them information while in the Army. He attempted to pass off false information to MI6, expressing a desire to be a double agent. Despite his amateurish efforts, he managed to collect sensitive military information for Iran, leading to his eventual arrest in January 2023 following a failed escape attempt. Khalife was later convicted of breaching the Official Secrets Act and the Terrorism Act.
GPT summary:  Brian Thompson, CEO of UnitedHealthcare, was fatally shot in New York, prompting initial condolences from Minnesota politicians. However, many social media users reacted with dark humor and outrage, sharing personal stories of b

In [ ]:
print(model_response_8)

Based on the two news articles, the public reaction was mixed:

1. In the context of Brian Thompson's death, the public reaction was one of dark humor and outrage, with many social media users sharing personal stories of being denied coverage by UnitedHealthcare. Many comments criticized the company's practices, and some even celebrated Thompson's death, linking it to the frustrations over the U.S. health care system.
2. In the context of South Korea's President Yoon Suk Yeol's declaration of martial law and his subsequent suspension, the public reaction was one of strong opposition and criticism. The declaration led to widespread protests, and the impeachment motion passed with significant support from the public, with polls indicating that three-quarters of South Koreans favored Yoon's removal.


In [ ]:
user_query_9 = 'what was the public reaction to the uhc shooting'
model_response_9 = model.generate_response(user_query_9)

BBBBBB
GPT-created query:  What was the public's reaction, particularly in terms of humor and critique, to the shooting of Brian Thompson, the CEO of UnitedHealthcare, as indicated by their shared memes, anecdotes, and personal experiences of denied claims?
Most similar context score:  0.81916255
Knowledge scores:  [0.8362973, 0.6027625, 0.48339033, 0.46851796, 0.4619391, 0.30288225, 0.27867293, 0.27106953, 0.26083016, 0.21711117]
# of factual results: 5
# of contextual results:  5
User-specific memory updated successfully!


In [ ]:
print(model_response_9)

According to the context, the public reaction to the shooting of Brian Thompson, the CEO of UnitedHealthcare, was a wave of dark humor and harsh critiques towards the healthcare industry and UnitedHealthcare specifically, instead of offering condolences. Many people shared memes, anecdotes, and personal experiences of denied claims to express their frustration and resentment towards the industry.


# Testing with Test Dataset

In [ ]:
import json
import evaluate
import time

In [ ]:
with open('dual_rag_dataset.json', 'r') as file:
    dataset = json.load(file)

In [ ]:
dataset

[{'topic': 'UHC CEO Shooting',
  'questions': ['Who shot the UHC CEO?',
   'Where was he shot?',
   'Where was the suspect found?',
   'How did people react?'],
  'answers': ['Luigi Mangione, a 26 year old Ivy League graduate, shot Brian Thompson, the CEO of UnitedHealthcare.',
   'Brian Thompson was shot in New York City in Midtown Manhattan outside of the Hilton hotel.',
   "Luigi Mangione was found at a McDonald's restaurant in Altoona, Pennsylvania with a fake ID, firearm, and manifesto after being recognized.",
   'People reacted with dark humor, critical commentary, sharing memes, jokes, and personal stories, criticism of the healthcare system, and anger and outrage towards health insurers.']},
 {'topic': 'South Korean President Impeachment',
  'questions': ['When was President Yoon impeached?',
   'Who impeached him?',
   'Why was he impeached?',
   'What was the public’s reaction?'],
  'answers': ['President Yoon Suk Yeol was impeached on December 14, 2024.',
   'South Korean l

In [ ]:
for conversation in dataset:
  print(conversation.keys())
  print(conversation.values())

dict_keys(['topic', 'questions', 'answers'])
dict_values(['UHC CEO Shooting', ['Who shot the UHC CEO?', 'Where was he shot?', 'Where was the suspect found?', 'How did people react?'], ['Luigi Mangione, a 26 year old Ivy League graduate, shot Brian Thompson, the CEO of UnitedHealthcare.', 'Brian Thompson was shot in New York City in Midtown Manhattan outside of the Hilton hotel.', "Luigi Mangione was found at a McDonald's restaurant in Altoona, Pennsylvania with a fake ID, firearm, and manifesto after being recognized.", 'People reacted with dark humor, critical commentary, sharing memes, jokes, and personal stories, criticism of the healthcare system, and anger and outrage towards health insurers.']])
dict_keys(['topic', 'questions', 'answers'])
dict_values(['South Korean President Impeachment', ['When was President Yoon impeached?', 'Who impeached him?', 'Why was he impeached?', 'What was the public’s reaction?'], ['President Yoon Suk Yeol was impeached on December 14, 2024.', 'South 

In [ ]:
pip install rouge_score

In [ ]:
# single_topic_scores = {}
rouge = evaluate.load("rouge")
# dataset = dataset[1:2]
# print(dataset)
for conversation in dataset:
    model = DualRagModel(logging='quiet')
    scores = []
    responses = []
    for question in conversation['questions']:
        # print(question)
        model_response = model.generate_response(question)
        responses.append(model_response)
    print("model responses: ", responses)
    print("answers: ", conversation['answers'])
    rouge_scores = rouge.compute(predictions=responses, references=conversation['answers'])
    print(rouge_scores)






query is:  Who shot the UHC CEO?
News search:  [{'source': {'id': None, 'name': 'TMZ'}, 'author': 'TMZ Staff', 'title': "Brian Thompson's Former Bodyguard 'Baffled' by Lack of Security", 'description': "A former bodyguard for Brian Thompson, the UnitedHealthcare CEO who was shot and killed in NYC, is in complete disbelief the exec didn't have security when he was murdered. Philip Klein, whose Texas-based security firm Klein Investigations…", 'url': 'https://www.tmz.com/2024/12/05/brian-thompson-former-bodyguard-baffled-by-lack-of-security/', 'urlToImage': 'https://imagez.tmz.com/image/69/16by9/2024/12/05/6950aef8eb714203835e77af120d1724_xl.jpg', 'publishedAt': '2024-12-05T18:27:24Z', 'content': "A former bodyguard for Brian Thompson, the UnitedHealthcare CEO who was shot and killed in NYC, is in complete disbelief the exec didn't have security when he was murdered.\r\nPhilip Klein, whose Texas… [+1527 chars]"}, {'source': {'id': None, 'name': 'Nakedcapitalism.com'}, 'author': 'Yves Smi

query is:  When was President Yoon impeached?
News search:  []
User-specific memory updated successfully!
query is:  Impeachment, President Yoon, political leaders, legal status, information
News search:  [{'source': {'id': 'rt', 'name': 'RT'}, 'author': 'RT', 'title': 'Coup attempt in South Korea: What was it all about?', 'description': 'The short-lived instigation of martial law was the result of a power struggle in Seoul and will likely lead to the president’s downfall Read Full Article at RT.com', 'url': 'https://www.rt.com/news/608734-south-korea-coup-attempt/', 'urlToImage': 'https://mf.b37mrtl.ru/files/2024.12/article/67508a2320302737af38b449.jpg', 'publishedAt': '2024-12-04T18:50:52Z', 'content': 'By\xa0Konstantin Asmolov, a leading researcher at the Center for Korean Studies at the Institute of China and Contemporary Asia of the Russian Academy of SciencesThe political crisis that culminated in … [+13140 chars]'}]
Knowledge database updated successfully!
User-specific memory u

query is:  When was Trump’s first conference after the election held?
News search:  []
User-specific memory updated successfully!
query is:  Who did he nominate as attorney general?
News search:  [{'source': {'id': None, 'name': 'Dnyuz.com'}, 'author': 'New York Times', 'title': 'Justice Dept. Girds for Test of Its Independence...', 'description': 'It was an early case of Donald J. Trump seeking retribution through the Justice Department. In the first year of', 'url': 'https://dnyuz.com/2024/11/29/justice-dept-girds-for-a-test-of-its-independence/', 'urlToImage': 'https://dnyuz.com/wp-content/uploads/2024/11/Justice-Dept-Girds-for-a-Test-of-Its-Independence.jpg', 'publishedAt': '2024-11-29T16:22:36Z', 'content': 'It was an early case of Donald J. Trump seeking retribution through the Justice Department.\r\nIn the first year of Mr. Trumps first presidency, Attorney General Jeff Sessions appointed a top federal p… [+8544 chars]'}, {'source': {'id': None, 'name': 'Americanthinker.com'}, '

query is:  Why have drones been in the headlines?
News search:  []
User-specific memory updated successfully!
query is:  Were they deployed by the US military?
News search:  [{'source': {'id': None, 'name': 'The New Republic'}, 'author': 'Edward Ongweso Jr.', 'title': 'AI Scams Are the Point', 'description': 'Why does it matter how we talk about artificial intelligence? Some, mainly tech firms and their useful idiots, maintain we are about to immanentize the eschaton (which translates roughly as: dissolve all of society’s problems). Others insist we are summoning …', 'url': 'https://newrepublic.com/article/188313/artifical-intelligence-scams-propaganda-deceit', 'urlToImage': 'https://images.newrepublic.com/225dcc2890ecff5a4d8efa31c87511bd54d4befb.jpeg?w=1200&h=630&crop=faces&fit=crop&fm=jpg', 'publishedAt': '2024-11-21T11:00:00Z', 'content': 'The militarys role is thoroughly neglected in this book. AI Snake Oil does not mention the RAND Corporation, an indispensable appendage of the Co

KeyboardInterrupt: 

In [ ]:
print(dataset)


[{'topic': 'South Korean President Impeachment', 'questions': ['When was President Yoon impeached?', 'Who impeached him?', 'Why was he impeached?', 'What was the public’s reaction?'], 'answers': ['President Yoon Suk Yeol was impeached on December 14, 2024.', 'South Korean lawmakers in the National Assembly impeached President Yoon Suk Yeol.', 'President Yoon Suk Yeol was impeached due to his declaration of martial law.', 'There were widespread protests. Thousands of protestors celebrated the result of the impeachment vote and viewed it as a victory for democracy.']}]


In [ ]:
queries_1 = ["Who shot the UHC CEO?", "Where was he shot?", "Where was the suspect found?", "How did people react?"]
outputs_1 = []
model = DualRagModel(logging='quiet')
# generate responses for each query
for query in queries_1:
    outputs_1.append(model.generate_response(query))
labels_1 = ["Luigi Mangione, a 26 year old Ivy League graduate, shot Brian Thompson, the CEO of UnitedHealthcare",
            "Brian Thompson was shot in New York City in Midtown Manhattan outside of the Hilton hotel",
            "Luigi Mangione was found at a McDonald's restaurant in Altoona, Pennsylvania with a fake ID, firearm, and manifesto after being recognized",
            "People reacted with dark humor, critical commentary, sharing memes, jokes, and personal stories, criticism of the healthcare system, and anger and outrage towards health insurers"
            ]

Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  UHC CEO, shooting, Luigi Mangione
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Suspect Location, Shooting Incident, New York City
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Reactions, Shooting, UnitedHealthcare
Knowledge database updated successfully!
User-specific memory updated successfully!


In [ ]:
rouge = evaluate.load("rouge")
rouge_scores = rouge.compute(predictions=outputs_1, references=labels_1)

print("ROUGE Scores:")
for key, value in rouge_scores.items():
    print(f"{key}: {value:.4f}")

ROUGE Scores:
rouge1: 0.4226
rouge2: 0.2225
rougeL: 0.3366
rougeLsum: 0.3443


In [ ]:
queries_2 = ['When was President Yoon impeached',
   'Who impeached him',
   'Why was he impeached',
   'What was the public’s reaction']

outputs_2 = []
labels_2 = ['President Yoon Suk Yeol was impeached on December 14, 2024.',
   'South Korean lawmakers in the National Assembly impeached President Yoon Suk Yeol.',
   'President Yoon Suk Yeol was impeached due to his declaration of martial law.',
   'There were widespread protests. Thousands of protestors celebrated the result of the impeachment vote and viewed it as a victory for democracy.']
model_2 = DualRagModel(logging='quiet')
# generate responses for each query
for query in queries_2:
    outputs_2.append(model_2.generate_response(query))


Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Yoon Suk-yeol, impeachment, South Korean lawmakers
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Impeachment, Yoon Suk-yeol, South Korea
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Impeachment, Martial Law, Protests
Knowledge database updated successfully!
User-specific memory updated successfully!


In [ ]:
rouge = evaluate.load("rouge")
rouge_scores = rouge.compute(predictions=outputs_2, references=labels_2)

print("ROUGE Scores:")
for key, value in rouge_scores.items():
    print(f"{key}: {value:.4f}")

ROUGE Scores:
rouge1: 0.4279
rouge2: 0.2900
rougeL: 0.3753
rougeLsum: 0.3753


In [ ]:
for output in outputs_2:
  print(output)


According to the text, President Yoon Suk-yeol was impeached by South Korean lawmakers, but the motion was passed with significant support from his own People Power Party (PPP), enabling the opposition to secure the necessary votes.
According to the text, South Korean lawmakers impeached President Yoon Suk-yeol.
According to the text, President Yoon Suk-yeol was impeached due to his controversial attempt to impose martial law, which was met with widespread protests and opposition from lawmakers. The opposition accused him of "inept semi-coup" and his
According to the text, the public's reaction to President Yoon Suk-yeol's declaration of martial law was one of widespread protest and anger. The text states that "significant public backlash" occurred, and thousands of protesters celebrated outside the National Assembly


In [ ]:
queries_3 = ['When was Trump’s first conference after the election held',
   'Who did he nominate as attorney general',
   'Who did he nominate for Secretary of State',
   'Who is he replacing as president',
   'What top executives has he met with or will meet',
   'What did he say in the conference']
outputs_3 = []
labels_3 = ['Donald Trump’s first press conference after the election was held at Mar-a-Lago.',
   'Donald Trump nominated Pam Bondi for attorney general.',
   'Donald Trump nominated Marco Rubio for Secretary of State.',
   'Donald Trump is replacing Joe Biden as president.',
   'Donald Trump will meet with Mark Zuckerburg, Tim Cook, Sundar Pichai, and Jeff Bezos.',
   "He discussed SoftBank's $100 billion investment plan in U.S. projects, the Ukraine war, border wall construction and potential legal action, New York Mayor Eric Adams' legal troubles, his transition process and meetings with business leaders, and drones in New Jersey."]
model_3 = DualRagModel(logging='quiet')
# generate responses for each query
for query in queries_3:
    outputs_3.append(model_3.generate_response(query))


Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Trump, attorney general, nomination
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Nominations, Secretary of State, Attorney General
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Donald Trump, Secretary of State, nomination
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Executives, Meetings, President
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Trump, executives, conference
Knowledge database updated successfully!
User-specific memory updated successfully!


In [ ]:
rouge = evaluate.load("rouge")
rouge_scores = rouge.compute(predictions=outputs_3, references=labels_3)

print("ROUGE Scores:")
for key, value in rouge_scores.items():
    print(f"{key}: {value:.4f}")

ROUGE Scores:
rouge1: 0.2883
rouge2: 0.1757
rougeL: 0.2668
rougeLsum: 0.2735


In [ ]:
queries_4 = ['Why have drones been in the headlines',
   'Were they deployed by the US military',
   'Are they a threat']
outputs_4 = []
labels_4 = ['There have been sightings of mysterious drones flying over New Jersey at night. There is no evidence of malicious activity or foreign involvement.',
   'No they are not military drones.',
   'The drones are not a threat to public safety or national security. Most of the sightings are misidentified legal aircraft and stars.']
model_4 = DualRagModel(logging='quiet')
# generate responses for each query
for query in queries_4:
    outputs_4.append(model_4.generate_response(query))

Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Were they deployed by the US military
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Drones, Threat, Sightings
Knowledge database updated successfully!
User-specific memory updated successfully!


In [ ]:
for output in outputs_4:
  print(output)

Drones have been in the headlines due to recent sightings in New Jersey and surrounding states, which have been attributed to a mix of misidentified legal aircraft, hobbyist drones, and stars, rather than a national security threat.
According to the text, the North Korean troops were deployed to Russia to support its military efforts against Ukraine, and the number is estimated to be around 12,000, with a potential to reach 100,000 within a year. There is no
According to the article, the investigations by the FBI and Department of Homeland Security found no evidence of a threat or foreign involvement in the drone sightings. In fact, the officials emphasized that many reported sightings likely involve misidentified commercial or military aircraft, and that


In [ ]:
rouge = evaluate.load("rouge")
rouge_scores = rouge.compute(predictions=outputs_4, references=labels_4)

print("ROUGE Scores:")
for key, value in rouge_scores.items():
    print(f"{key}: {value:.4f}")

ROUGE Scores:
rouge1: 0.2250
rouge2: 0.0435
rougeL: 0.1701
rougeLsum: 0.1701


In [ ]:
queries_5 = ['What player left mid game in NFL',
   'What team and position does he play',
   'How long was he suspended for',
   'Why was he suspended',
   'How did his teammates feel about it']
outputs_5 = []
labels_5 = ['De’Vondre Campbell was suspended from the NFL.',
   'De’Vondre Campbell is a linebacker for the San Francisco 49ers.',
   'De’Vondre Campbell was suspended for 3 games due to conduct detrimental to the team.',
   'He refused to go into a game after losing his starting spot.',
   'His teammates were angered, bewildered. They believe his actions were selfish and hurt the team.']
model_5 = DualRagModel(logging='quiet')
# generate responses for each query
for query in queries_5:
    outputs_5.append(model_5.generate_response(query))

Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  What team and position does he play
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Suspension Duration, Team Position, Context
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Suspension Reason, Duration, Context
Knowledge database updated successfully!
User-specific memory updated successfully!
query is:  Teammates' feelings, suspension reason, context needed
User-specific memory updated successfully!


In [ ]:
rouge = evaluate.load("rouge")
rouge_scores = rouge.compute(predictions=outputs_5, references=labels_5)

print("ROUGE Scores:")
for key, value in rouge_scores.items():
    print(f"{key}: {value:.4f}")

ROUGE Scores:
rouge1: 0.0897
rouge2: 0.0070
rougeL: 0.0747
rougeLsum: 0.0830
